In [1]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill, Alignment
from openpyxl.utils.dataframe import dataframe_to_rows
from datetime import datetime
import locale
import tkinter as tk
from tkinter import filedialog, Label, Entry, Button

# Configuración regional
locale.setlocale(locale.LC_ALL, 'en_US')

# Función para procesar el archivo de Excel y crear nueva hoja con DataFrame procesado
def process_excel_file(file_path, sala, exchange_rate):
    df = pd.read_excel(file_path, sheet_name='Dólar Estadounidense', header=7)
    df = df.drop(df.columns[0], axis=1)
    columnas_especificas = ['Fecha Final', 'Cod. Máquina', 'Juego', 'Nro. Serie', 'Coinin', 'Win', 'Avbet x Game', 'Games Played']
    df = df[columnas_especificas]
    df['Cod. Máquina'] = pd.to_numeric(df['Cod. Máquina'], errors='ignore')
    df['Nro. Serie'] = df['Nro. Serie'].astype(str).str.lstrip('0')
    df['Fecha Final'] = pd.to_datetime(df['Fecha Final']).dt.strftime('%d/%m/%Y')
    df.insert(1, 'sala', sala)
    df.rename(columns={'Cod. Máquina': 'cod_maquina', 'Juego': 'juego', 'Nro. Serie': 'serie', 'Coinin': 'coin_in', 'Win': 'net_win', 'Avbet x Game': 'average_bet', 'Games Played': 'game_played'}, inplace=True)
    df = df.iloc[:-1]
    # Aplicar condiciones adicionales a las columnas average_bet y game_played
    df.loc[df['average_bet'] < 0, 'average_bet'] = 0
    df.loc[df['game_played'] < 0, 'game_played'] = 0
    df.loc[df['average_bet'] == df['coin_in'], 'average_bet'] = 0
    df.loc[df['game_played'] == 0, 'average_bet'] = 0
    df.loc[(df['game_played'] > 310000), ['game_played', 'average_bet']] = 0
    df = df.sort_values(['coin_in', 'game_played'], ascending=[True, True])

    # Convertir exchange_rate a float
    exchange_rate = float(exchange_rate)

    # Aplicar factor de conversión
    df['coin_in'] *= exchange_rate
    df['net_win'] *= exchange_rate
    df['average_bet'] *= exchange_rate

    # Renombrar columnas
    nuevos_nombres = {
        'Fecha Final': 'fecha',
        'Cod. Máquina': 'cod_maquina',
        'Juego': 'juego',
        'Nro. Serie': 'serie',
        'Coinin': 'coin_in',
        'Win': 'net_win',
        'Avbet x Game': 'average_bet',
        'Games Played': 'game_played'
    }
    df.rename(columns=nuevos_nombres, inplace=True)

    # Crear una nueva hoja en el archivo de Excel y escribir el DataFrame procesado
    archivo_excel = load_workbook(file_path)
    hoja_nueva = archivo_excel.create_sheet('Hoja-Dolar')
    hoja_activa = archivo_excel.active
    for fila in dataframe_to_rows(df, index=False, header=True):
        hoja_nueva.append(fila)
    fuente = Font(name='Arial', size=9)
    relleno_encabezado = PatternFill(start_color='D3D3D3', end_color='D3D3D3', fill_type='solid')
    alineacion = Alignment(horizontal='left')
    archivo_excel.save(file_path)

# Función para abrir el cuadro de diálogo de selección de archivo y obtener los valores de sala y exchange_rate
def open_file_dialog():
    root = tk.Tk()
    root.title("Interfaz para Procesar Archivos")

    def process_file():
        sala_value = sala_entry.get()
        exchange_rate_value = exchange_rate_entry.get()
        file_path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx")])
        if file_path and sala_value and exchange_rate_value:
            process_excel_file(file_path, sala_value, exchange_rate_value)

    Label(root, text="Ingrese el valor de sala:").pack()
    sala_entry = Entry(root)
    sala_entry.pack()

    Label(root, text="Ingrese el tipo de cambio:").pack()
    exchange_rate_entry = Entry(root)
    exchange_rate_entry.pack()

    Button(root, text="Seleccionar Archivo", command=process_file).pack()

    root.mainloop()

# Llamar a la función para abrir la ventana de selección de archivo, valor de sala y exchange_rate
open_file_dialog()
